<a href="https://colab.research.google.com/github/souravs17031999/private-ai/blob/master/Final_federated_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT - XII

## OBJECTIVE : IMPLEMENTING FEDERATED LEARNING ON MNIST DATASET

We have already implmented federated learning on toy dataset , now it's time for some real applications.
Let's first install pysyft.

In [0]:
pip install syft

Start by importing all packages required for torch and pysyft

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
import syft as sy
hook = sy.TorchHook(torch)
# let's create two virtual workers who will hold the data while training the model locally
bob = sy.VirtualWorker(hook, id = "bob")
alice = sy.VirtualWorker(hook, id = "alice")

W0711 04:42:56.079641 139812741711744 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0711 04:42:56.095262 139812741711744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



It's time to load the data and apply our transforms !

We first load the data and transform the training Dataset into a Federated Dataset using the .federate method: it splits the dataset in two parts and send them to the workers alice and bob. This federated dataset is now given to a Federated DataLoader which will iterate over remote batches.

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [0]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

In [20]:
len(test_loader)

157

In [0]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [26]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.307303
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.311168
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.192290
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.152380
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.086846
Train Epoch: 1 [3200/60032 (5%)]	Loss: 1.989702
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.851289
Train Epoch: 1 [4480/60032 (7%)]	Loss: 1.685259
Train Epoch: 1 [5120/60032 (9%)]	Loss: 1.537941
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.258122
Train Epoch: 1 [6400/60032 (11%)]	Loss: 1.041625
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.869986
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.786288
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.612632
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.466813
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.752732
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.530697
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.524642
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.389656
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.659216
Train Epoch: 1 [12800/60032 (